In [27]:
import requests
from bs4 import BeautifulSoup
from google.colab import files
import csv
import os
import re
import pandas as pd

In [28]:
# Lista para armazenar as URLs encontradas
URLs = ['https://www.nfl.com/stats/player-stats/category/passing/2024/post/all/passingyards/desc',
'https://www.nfl.com/stats/player-stats/category/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAGQAAABlAoMIAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNVFExSWl3aU16SXdNRFF6TkRFdE5USXpOUzA0TnprM0xUTmlPV0V0TUdReU56RXhZekJpWXpJM0lpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAMgAAADJAdpAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpOakVpTENJek1qQXdORFEwWmkwME1qTXdMVEl6TmpBdE9HVXhOQzFrWW1KbE1XUmxaR1V4TWpBaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAASwAAAEtANgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNaUlzSWpNeU1EQTBZVFExTFRRMk1qWXRPVEk0Tnkxak5EVTBMVFEzT1dJNU5ESmpZbVF5TmlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAZAAAAFsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmpOREV0TldFd05DMHpOekUwTFdObVlqY3RPRGN5WWpJd1lqQmpaVEV6SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/post/all/rushingyards/desc',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAAGQAAABlAijgAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRNemtpTENJek1qQXdOVFExTWkwME1URXhMVFl5TnpjdFpUZzVNaTB6WVdJd01qWXhNbVUwWXpJaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAAMgAAADJAe1AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBNemNpTENJek1qQXdOVE0wT1MwMFpURTRMVFk1TVRrdFpERmxOaTB3TURObU5EWXpNbUpoT0RjaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAASwAAAEtAcJAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlOalVpTENJek1qQXdOR0UwWmkwMFpURTJMVFUwTmpJdE16bGhOQzB6WWpSbVlUYzBNMk0yTm1ZaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAAZAAAAGRAZYAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhOeklpTENJek1qQXdOR1EwTXkwME9UVXdMVEF5T1RndE0yRmpZeTB3TkROa056QmtNelJtT1dRaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAAfQAAAH1AWoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNRFlpTENJek1qQXdOVEkxTlMwME5ETXlMVEV5TVRZdFlUbGlaUzB5T0RVd01tTXdPRFEzWlRRaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAAlgAAAJZATQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFPQ0lzSWpNeU1EQTBaRFE1TFRSbE5qQXRORFl4TUMxa1l6azRMV1JqWkdSbFlUbGlZakZtTmlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAArwAAAK9AQwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpPQ0lzSWpNeU1EQTBZelUxTFRRMU5UZ3RNakkzTlMxaE56UmlMVGcwWlRkak1qWXhNRGcyWlNJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAAyAAAAMdANwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNeUlzSWpNeU1EQTBaRFExTFRVNU1UY3RNVFl3TmkwME1UazNMVGhtTkRZM056bG1ZamxpTnlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAA4QAAAN9AKgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNeUlzSWpNeU1EQTFORFF4TFRVNU56WXRORGcyT0MxaU9ERmtMV1JtTXpGbVltSTNabVUzTWlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAAA-gAAAPhAHAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STNJaXdpTXpJd01EUTBOREV0TlRZNE1pMHlOVFV5TFdKaFpqTXROalUzT1dNeE5EYzBPV1l4SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAABEwAAARNACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpJaXdpTXpJd01EUXlOR1l0TlRrek55MDRNelV6TFdJNU5ERXRaR1poTnpFeU1tSmhNVFppSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAABLAAAASQ_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVXpORGd0TkRVME5TMDRPRGt6TFRSbE9HSXROVGN5TkRNd01tUmxNRGc0SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/rushing/2024/REG/all/rushingyards/DESC?aftercursor=AAABQQAAAUDACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXRNeUlzSWpNeU1EQTBZelJtTFRSbE1ERXRNelEyTXkxaE0yVTVMV1k1TldVNE1EQmpPR0ppWVNJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/post/all/receivingreceptions/desc',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAAGQAAABhAU4AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STNPQ0lzSWpNeU1EQTBZalE1TFRVME16Z3RPREk1TUMxbFpXVmtMV001WXpsalkyRTJNbVZqTVNJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAAMgAAADFATwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJNaUlzSWpNeU1EQTBaRFF6TFRRek5ERXROekV4T0MwM05EVmpMVEpqT1RjMU5qSTVNbUpqT0NJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAASwAAAEdASYAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFNU0lzSWpNeU1EQTFOelE0TFRRNU16a3RNekV4TUMwNU1ETmxMVFppTm1NNE5qaGtaRE5oWWlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAAZAAAAGRARYAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBNeUlzSWpNeU1EQTBNalJtTFRVMU5EZ3RNamd5TXkxaU9UTm1MVE15WmpkaU5qaGlNR1JrTXlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAAfQAAAHxAQoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpOeUlzSWpNeU1EQTBOalJtTFRVeU1qTXROREF5TnkxallqbG1MVGxtTW1GbE9EQXpZMk5rTlNJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAAlgAAAJBAQAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpNaUlzSWpNeU1EQTFNelE0TFRRMU5Ea3ROVGMxTkMwMU5qazNMVEpoT0RNeE1tVmtZelE0T1NJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAArwAAAK5AOgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlOaUlzSWpNeU1EQTBZVFJtTFRRNE56RXRNVEE1TkMwMU9HUTVMV05oTkdWbU56a3lNbVk0WXlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAAyAAAAMVANAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNQ0lzSWpNeU1EQTFNalJtTFRReU56TXROakUzTnkwMll6QmtMV1EzT1dWaU9XUXdNRGRrTVNJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAA4QAAAOFAMAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhOaUlzSWpNeU1EQTBORFExTFRSa016RXROemM1TkMwM1pEVXdMV1l5TUdFMlpETmxNelZsWWlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAA-gAAAPdAKAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNaUlzSWpNeU1EQTBOalF4TFRVeU5qRXROVGMwTUMwMU1HSTRMVE13TldSaU1HSTNaVEl6T0NJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAABEwAAARJAJAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNQ0lzSWpNeU1EQTBOalJqTFRSbU1qUXROVEEyTVMwME1XWTVMVFl5TVdNeFpqUTRNVFF5TUNJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAABLAAAASdAIAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRJaXdpTXpJd01EUTROVGt0TkRFME1pMHdPVEUyTFRZNFpEQXRabVl6WXpVME4yUmxNelEySWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAABRQAAAT9AGAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJJaXdpTXpJd01EUTJOR1l0TlRJek9TMHlOakU0TFRjeFpHVXRPV0l5TW1Nek9UTTVOakUzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAABXgAAAU5AFAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFJaXdpTXpJd01EVTVORFV0TkRJME1TMDFNamt4TFdOak5tVXRZek0zTVRFek5XUmlPVEF3SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAABdwAAAWlACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpJaXdpTXpJd01EUTROR1l0TkdRMk5TMDFOVFU0TFRRMU5tSXROMkk1Wm1FNVpqazNNRGt3SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAABkAAAAYhAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EVXdORGt0TkRVeU5DMDFORGM0TFRFeE5XTXRPR0ZsTVRGa1ltVmxabUUySWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAABqQAAAZo_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUTNOR1l0TkRZeU1TMDVOak0yTFRjek5UUXRaREpsWWpZek5qVmtNalUzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAABwgAAAZo_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVXdOVEl0TlRrME5TMDBOVFF4TFdWbE56Y3RZamszTVRVMk56TTVNVEE1SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAB2wAAAdcAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXlORFV0TlRJME5TMDJNRFV3TFdJNVpEZ3RPR0l5T0Rnd01EWmlaRFEySWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/receiving/2024/REG/all/receivingreceptions/DESC?aftercursor=AAAB9AAAAdcAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXpOVFV0TkdNMk5TMDBPRFF4TFRZMllqRXROelF5TjJJeFlXTTRPVFZrSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/post/all/defensiveforcedfumble/desc',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAGQAAABhAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EUXhOVEl0TkdReU1TMDRPVEExTFdaaFlXRXRNMlkwWkRrME16QTNaRFl3SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAMgAAABhAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EUmhOR1l0TkdVMU55MDVPVE00TFdFMU5UY3RPV0U0TlRrNU16QmpNV0ppSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAASwAAABhAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EVTJOREV0TkdVMk5TMDJOakEwTFRNMVpXSXROemt3T1RReE5XVTBNVFUxSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAZAAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUXlORGt0TlRNeU5DMDBPVFUxTFRjNE1qZ3RZMlppT1dNd1lUWmxZbUZsSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAfQAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUXpOR1l0TkdVME5pMHhPVGs1TFRVNVpUa3RNalV6WTJWa1pXVmpNMkUxSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAlgAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUTJOR010TkdZM05TMDRNRFl3TFRWaE5tUXRaRE5oWVdKak0yTXpNbVV5SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAArwAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUTROVFV0TkRZd09TMHdPRFEwTFRZNFlqRXROall4TnpRME1XVmhZVFUzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAyAAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUmpORFV0TlRjME1TMDVOVGszTFRZMllXSXRZakU0WkRWbVlqWTFPV0UxSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAA4QAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUmxORFV0TlRjM01TMDBPRGcxTFRneVl6Y3RORGs1TURZeE1HTTNZMlUzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAA-gAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVXlORFV0TkRFMk5pMDBNVFV4TFdJM04yUXROREptT1RabE0yWTBZekUzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABEwAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVXpOVFF0TkdFMk1TMDFNekV3TFdabU9ERXRaVFUzWlRrNU16QTJZVEk1SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABLAAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVTNORGt0TkRjeE9DMDNNVEUyTFRrMFptSXROV015WW1GbVlqY3pZMlkxSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABRQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXhORFF0TkRFMU1TMDVPVEExTFRjd1pESXRPVEk0TnpjeU5UZGlNRFkwSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABXgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXhOR1V0TkRjeU9DMHdPVEV4TFdOaFpESXRaVGhqTjJOaE5ETTBPVFUySWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABdwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXlOREV0TlRJMU1pMDRNekU0TFdRNU1UWXRNMlJqWVRjMVlqUmhabUZpSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABkAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXlORFV0TlRJM055MDVOVGc0TFdFd05XVXROakZpWldZek5qRmxZemxqSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABqQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXlOR1l0TlRjM055MHdNREl5TFdaaE9HTXRZakU0WTJOaE0ySmtOMlkySWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABwgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXlOVEl0TkdZMU5pMDBPVFEyTFRaa1pXTXRNelpoTnpNM1lqTXlPV1UwSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAB2wAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXpOREV0TkdReU1pMDNPVEl5TFdaaVpXVXRZVGhoWmpZNE5USTRNbVV6SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAB9AAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXpOR010TkRFME1DMDVNalV4TFRKbU9EZ3RaRFExTkdJNFlXVTJNek5pSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACDQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXpOR1l0TlRnMU1pMDBPRGd3TFdOa05HWXROV1U0WkdabFlqWmhaVFpqSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACJgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTBOREV0TlRZMU1TMHpPVGc1TFRjek5URXROV1ZsWVRZM01UazNZelkzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACPwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTBOR1l0TlRVeU5TMDVOVFV5TFRWa1l6a3RabVJoTjJNMVlUbGlNRE5rSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACWAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTFOR010TlRVeE55MDNPVGd4TFRKa1lqTXRNamt5WVRNd1pqUTVNMlV5SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACcQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTJOR010TkdZd01DMHdNREF3TFdZeE5qUXRPREUzTnpsbU9XSTRORGd5SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACigAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTNOREV0TkRrMk1DMDJPVFkwTFRjM01ESXROamt5TmpVMlptRmtPRFl4SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACowAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTNOR1l0TkRRMU5DMDVNemt6TFRabFpXTXRNalF4T0Rsa1pEWTJOalF4SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACvAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTNOVFV0TlRreU15MHdOekUyTFRsak1Ua3ROV1ExTWpJeFpqZ3pPV1U0SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAC1QAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTROREV0TlRJME5TMHdOVFF5TFRZME1HSXROV1JqTlRjeU9EUTFZalpqSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAC7gAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTRORGt0TkRNME5pMDFPREl5TFRkbU9USXRObUppTm1FelkyRTJNR1UwSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADBwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTROR1l0TkdZeU9TMDROelV6TFRZNVptTXROVFppTmpoaU0yRTVZakUzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADIAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTVOR1V0TkRjMU5DMDFORE16TFRZM01ESXROV0kwWXpVNVlXTmhNMkU0SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADOQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmhORFV0TkRZME1pMHpOVE00TFRZeU5EQXRPREZrWXpGa1pqUmpNekpsSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADUgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmhOR1l0TkRnek5pMHlOalF4TFRrNU1ETXRaamMwTVRkak1URTJPRFUzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADawAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmhOR1l0TkdVM05pMDROemd5TFRFMk0yTXRPR1E1WkdJeU9XWTFZMkU1SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADhAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmlORGt0TlRJeU5TMDNPVEU0TFdVME1ESXRPREZtWldWa00ySmtZV1ppSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADnQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmpOREV0TldFd05DMHpOekUwTFdObVlqY3RPRGN5WWpJd1lqQmpaVEV6SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADtgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmtOREV0TkdNMk5DMDNORGswTFRnMk1UQXROMkk1TjJJNFpHTmhZV0kxSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADzwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmtORE10TkRNME9TMDFNVGcxTFRFMlpURXRZV000TW1FeVptTTFOalV4SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAD6AAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmtORFV0TkdNMU55MDFORE14TFRFd05qRXRZak14TkdFME1qYzJNbUptSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEAQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmtOR1l0TkdZM01DMDJOVEU0TFdWbU5UQXRORFU1TWpVellXVTJZekF6SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEGgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmxORFV0TkRFME55MDFPRFE0TFRrek16Y3RNR1F5WlRZd056STVOamd4SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEMwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUm1OR0V0TkRFME1TMDFNamt4TFRsak5tUXRPRFE1TWpFM1lUTXhNbUl6SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAETAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXdOREV0TlRreU9DMHhNakl5TFRaaFptRXROREZsWXpreU5UWXlOMkUwSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEZQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXdOR1l0TlRreE55MDNPVGd4TFRRNVpUTXRORFV3TkdReE1ETTJPV1ppSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEfgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXlORFV0TkRrek1DMDJOekl6TFdNME5tWXRNRE5rWkdFM05UTXlZV05rSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAElwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXlOR1l0TkRJek9DMDBPRGc0TFRZMVltSXRZbVEzWVRJek1qRTNaR1F6SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEsAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXpOREV0TkdVd05DMDNOell5TFRRM056VXRPVGRsWkdJM056RTVNbVEzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEyQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXpORGd0TkRVNE1DMDJNak01TFRJMU1HSXRaR1poT1RFd1pqWXlOekprSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAE4gAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXpOR1F0TkRrM015MDVNRFF5TFdFeVpUVXROMkUyTVRGak0yTXpNakUwSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAE-wAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXpOVFF0TkRrek9TMDRPRFUyTFRrNFpHTXRORGsyTXpReFlXVXpOVEEySWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFFAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTBOREV0TlRZek9DMHdNak00TFdRMlkyTXRNalZoWVRFM1lqRTRZekJrSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFLQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTBOR1l0TkdNeE5DMDRPVEUwTFdSbE9URXRZMlExTkdRMU9XUmxNbUV5SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFRgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTFOVEl0TkRJeU1pMDJOVEl4TFRsbE1UQXRZalV6TlRjell6VmpaR1UwSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFXwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTNOREV0TlRJd055MDJPREl3TFRJMk16UXROREkzTkdJMk5ESTVaVEZoSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFeAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTNORGd0TkRrMk1pMHlOek14TFRGa09USXRZMlU0TUdRMU1USmtZak0wSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFkQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTNORGt0TkdNMU9TMDNNek14TFdReVpUa3RaRFJsTURsaVpUSTRZekk0SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFqgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTNOR1l0TkdZMU1TMDJNelU1TFRneU4yRXRNV1prWlRoa1l6RTNPVGd5SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/post/all/defensivecombinetackles/desc',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAGQAAABlAXoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNakl1TUNJc0lqTXlNREEwTkRReExUVTJNREl0TmpZMU5DMWxaR0k0TFdGaE5tUmhOVFkzTXpJMU15SXNJakl3TWpRaVhYMD0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAMgAAADJAWcAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNRE11TUNJc0lqTXlNREEwWkRRMUxUVTBNemd0TXpRME1pMHlabU5pTFRrMVltWTVOV0kxTWpVNE1TSXNJakl3TWpRaVhYMD0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAASwAAAEpAVgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRPQzR3SWl3aU16SXdNRFJrTkRNdE5HSTFOQzA0TnpZMUxXRTJaRGt0TlRWallUQTFOVEF5WlRaaUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAZAAAAGJAVAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRNQzR3SWl3aU16SXdNRFJrTkRrdE5HTTFPQzA0T1RjeUxUTmpZMkl0TWpJeFlqUmhNV015TkRSbUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAfQAAAHdAUsAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STNOUzR3SWl3aU16SXdNRFUzTkRrdE5HTTNOQzAzTVRZNUxXWXlOVEV0WlRZNVl6UXdaRFExTnpZMUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAlgAAAJVAUUAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJPUzR3SWl3aU16SXdNRFJoTkdZdE5HVTBNQzB6T1RNMExUTXhZVFV0TWprd1pEQmtaV1l4WVRSaUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAArwAAAKxATwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJNaTR3SWl3aU16SXdNRFV6TkRrdE5HUXlOQzA0TmpneExXVTNaVGt0TVRoalpqSmhPRFZpT0dJNUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAxwAAAMZATQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFPQzR3SWl3aU16SXdNRFF6TkdZdE5HWTJNUzAzTVRNNUxUaG1aVGd0TXpjeE9UZGxZVFkzTUdaaElpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAA4AAAAN9ASwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFOQzR3SWl3aU16SXdNRFEwTkRrdE5ERXdOUzB4T1RFeExUVTJNbU10TjJRM05tWTBaVFE1WWprNUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAA-QAAAPNASYAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFNUzR3SWl3aU16SXdNRFV6TlRjdE5EVXdNeTAyTVRreExUTXhNakV0TVRFeFlqVXhaV1ExWkRGaElpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABDwAAAQ1AR4AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBOeTR3SWl3aU16SXdNRFEzTkRFdE5USTFNaTA0T0RrMkxUSmpNV1F0TldGaFptVmpOMlEwTjJVNElpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABKAAAASNARoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBOUzR3SWl3aU16SXdNRFJpTkdZdE5EZzBOaTB4TkRNMUxUTXhPVEF0TUdVMllUaGpaR1JsTjJFd0lpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABPgAAAT1ARQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBNaTR3SWl3aU16SXdNRFEwTkRrdE5EY3lNQzB5T0RRMUxXWTJPV1V0TWpBeE1EWTBPVGxtTm1ZMklpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABUwAAAUtARIAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBNUzR3SWl3aU16SXdNRFV3TkRFdE5Ua3hOaTA0T1RnMkxXUTVabUl0WkRWallqRmtNVGRsWWpReUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABaQAAAWdAQ4AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpPUzR3SWl3aU16SXdNRFExTlRBdE5EVTBNUzAxTWpreExUa3pOamt0WWpsa01HTXhOekJoTlRZMUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABfQAAAXdAQoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpOeTR3SWl3aU16SXdNRFJoTkdZdE5HVXhOaTB5TlRrMUxUUmhNV1F0T0Rsa1lUQXlZMlV3TkRsaUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABkgAAAYhAQYAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpOUzR3SWl3aU16SXdNRFZpTkdVdE5HWXdNQzB3T0RFd0xUUTBPR1V0TURRd05qQmpOelprTWpRMklpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABqQAAAaFAQAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpNaTR3SWl3aU16SXdNRFUwTkRFdE5Ua3hOQzAyTVRrekxUWTVPRGd0WkdRME5ESmtZbVkzWmpZMUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABvwAAAb5APQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlPUzR3SWl3aU16SXdNRFF6TkdZdE5HTXlNUzAwTXprM0xXWTNOVGN0TkRabVlqazFaRFl6WmpWbElpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAB1wAAAcpAPAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlPQzR3SWl3aU16SXdNRFV3TkRFdE5UQTNNaTB6TVRjMExXRXdaakl0Tm1Jek5UVmtOamN5TnpZd0lpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAB7wAAAeZAOgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlOaTR3SWl3aU16SXdNRFE0TkRFdE5USTJOQzAzTmpBeExUSmpabVV0WkRNek5HTTVNR1UwTkRGbUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACBwAAAf5AOAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlOQzR3SWl3aU16SXdNRFJrTkRrdE5UUXhOQzA1TnpRNUxUVXlOall0WVdVM05UZzFNak15TlRRd0lpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACIAAAAhpANgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNaTR3SWl3aU16SXdNRFE0TkRVdE5USXdOQzB5TVRBM0xUazRPV0l0WWpsaE1HVmlaakV3T0RRMUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACOQAAAjVANAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNQzR3SWl3aU16SXdNRFJrTkRFdE5EUXpOQzAzTlRnMUxURmxaVEF0TURNNVlqVmhPRFl6Wm1RNUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACUQAAAkJAMwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhPUzR3SWl3aU16SXdNRFUzTkRndE5Ea3pPUzA1T1RneUxXUmhabVV0T0dGaFkyVTBZbUV3WkRRNElpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACagAAAl1AMQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhOeTR3SWl3aU16SXdNRFJrTkRrdE5HTTFPUzAzTmpVMExUZzJNRGd0TmpJeU1EVXhZalV6WW1Reklpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACgwAAAn9ALgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhOUzR3SWl3aU16SXdNRFJoTkdZdE5HVTBOQzA1TnpFMUxXUmpPRE10WmpWbFlqYzNZelF6WlRWaklpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACnAAAAo5ALAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhOQzR3SWl3aU16SXdNRFJoTkRVdE5EWXpOUzB3TXprNUxXUmlabUl0TXpjM05XRTJZV00zT1RWaklpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACtQAAAqdAKgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNeTR3SWl3aU16SXdNRFJsTkdFdE5HWXpOaTAzTWpFMExXVmhNMkV0TlRoa1lXRTFaak5sWTJJMUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACzgAAAr5AKAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNaTR3SWl3aU16SXdNRFJoTkdZdE5EZ3lNQzB5TVRnNUxUWmxNall0TVRBM05tSTBPVGRqTVdReUlpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAC5wAAAtlAJgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNUzR3SWl3aU16SXdNRFV5TkRrdE5HTXhOQzAyTmpRNExUSXhORFF0TUdVeVpUVmhZbVExTURJd0lpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADAAAAAv1AIgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STVMakFpTENJek1qQXdOREkxTlMwMU5EUTVMVEExTWpBdE9UWTNaUzAxTmpjM09UWTBOVEJpTWpnaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADGQAAAxVAIAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRMakFpTENJek1qQXdORGMwTVMwMU9UVTBMVFU0TnpjdE5qUmpNUzB3TjJFM1lqZGhPRFpsT0dZaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADMgAAAyxAHAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STNMakFpTENJek1qQXdOREkxTWkwMFpqWTVMVFV3TURNdFpUY3lZaTAxTURKaFpUbGhZVEUxTlRnaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADSwAAA0tAGAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJMakFpTENJek1qQXdOREUwWXkwMFl6Y3dMVEkxT0RJdE5HTmpZUzB4WVRJd056QmpPV0V3T1RBaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADZAAAA0tAGAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJMakFpTENJek1qQXdOVE0wWkMwME9UZ3lMVFkxTURNdFlUTTVNaTFrWXpBME56aGtPR0pqWWpNaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADfQAAA2tAFAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFMakFpTENJek1qQXdORGcwTVMwMU1qQTBMVGt5T1RjdFl6YzJOQzB4Tnpnd1lqQTBPRGRqT1RNaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADlgAAA5VAEAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBMakFpTENJek1qQXdOREUwTkMwME1UUTVMVGt5TlRNdE5ERXpZUzA0TW1aa05EUTNOalZtWWpnaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADrwAAA5VAEAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBMakFpTENJek1qQXdOR0UwWmkwMFpUVXhMVGd6TXpVdE4yUTJZUzFsTm1JM1ptWXlaVEE1WldNaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADyAAAA5VAEAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBMakFpTENJek1qQXdOVGMwT0MwME9UZ3dMVGMyT0RRdFpEZ3hZaTFrTkdaa09ERmlNRFJpWTJJaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAD4QAAA8xACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpMakFpTENJek1qQXdORE0wWmkwMFpUUTJMVEU1T1RrdE5UbGxPUzB5TlROalpXUmxaV016WVRVaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAD-gAAA8xACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpMakFpTENJek1qQXdOR1EwTVMwMU5ERTVMVFUyTkRJdE5qWXpOaTB4WlRSall6SXhaVGMwTW1RaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEEwAAA8xACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpMakFpTENJek1qQXdOVFEwTVMwMU9UWXhMVEExTlRNdE9URTNZeTB6WVRVMlpUVmhOR015Tm1NaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAELAAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdOREkwTVMwMU5EVTBMVFV3TmpVdFpHSmlPUzA0TlRReFpUZGtaalExT0dFaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAERQAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdORFUxTWkwMU5EZ3dMVEE1TWpBdE16SmhOaTB4TmpWaU5XRTNOemd4TkdVaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEXgAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdORGswWlMwME56TTRMVEkyTmpFdFltRTNOeTA0WXpNMU5EUTROakk1TXpnaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEdwAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdOR1kwWXkwME1UWTFMVGt6TWpVdFl6STFPUzB3TVdFMU1qRXlNREpsTW1ZaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEkAAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdOVE0xTlMwMFpURXpMVEUyT0RBdFkyUTNaUzB5TURsbVptUmhOamMyTlRNaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEqQAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOREUwWXkwMFl6QTJMVE00TVRZdFlUazVOUzB5TVRZeE5HSTVPRFEzT1RnaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEwgAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOREkxTWkwMFpqRXpMVEk1T0RndE56QXhZaTA1TUdJMVlqQTVOak0yWTJFaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAE2wAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdORFEwT1MwME56SXhMVGc0T1RVdFptTXlNeTFrTW1KbU5HTTVNakZpWkRRaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAE9AAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdORGMwT1MwMFl6STNMVEl4T0RFdE1XUmxaQzFrWkRRNU5qUTRNMkUwTUdRaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFDQAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdORGcwWmkwMU1ERTJMVEUxTURBdFltWmlaUzFqT0RkaU1qa3lZall6T1dNaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFJgAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOR0kwTVMwMFpESTBMVFF3TlRndE5XRXpZaTB3TVdKa016bGxObUV5TldRaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFPwAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOR1EwTXkwME16QTFMVGMyT0RrdE9Ea3laQzB3TlRabVltUXpOVEk0TVdRaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFWAAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOR1kxTWkwMVlUUXhMVFV5T1RFdFltVmlPQzFsWW1ZME1Ua3dNekJoTVRnaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFcQAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOVEkwWmkwME1qSTBMVEF3T0RndFlqQmlPUzB5WTJNd05UUmlZVGN4TjJRaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFigAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOVE0xTkMwME9UWXlMVGd3TmpjdFlURmxOQzAyWkRobVkyVTNNak0xWVRnaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFowAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOVGMwT1MwMFl6SXdMVFF6T1RNdE9UY3hOUzFrWldFME1tVmlNalJrWXpZaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/post/all/defensiveinterceptions/desc',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAGQAAABNACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpJaXdpTXpJd01EUTBOR1l0TkRRek5DMHhORFl3TFdJek56Z3RPVEV4Tnpaa056QTBZemMzSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAMgAAACZAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EUTNOVEl0TkRreU9TMDJOVFkxTFRkbU5UUXROVGxtWm1NM016VTNNak5tSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAASwAAACZAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EVXpORGt0TkdReU5DMDROamd4TFdVM1pUa3RNVGhqWmpKaE9EVmlPR0k1SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAZAAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUXlOR1l0TkdNMk55MDRNekF4TFdOa05qWXRNbUkzWlRRMU5ETTVNakpoSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAfQAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUTFOR010TkdNM05DMDRNRFUzTFRnek5UZ3RNR013WWpGak1qaGpPR1F4SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAlgAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUTROR1l0TlRJeE1pMHpOVGsxTFdWaFltWXRObVkyWmpOalpHUmtZelF3SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAArwAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUmtOVFV0TlRJMU5pMHpORGt3TFRVM05HRXRNV0k0WVRnMllXRTJPVEZtSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAyAAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVXlOR1l0TldFd05DMDVPRE0wTFRFNU5tRXROelEzTlRNM1pHRm1aak16SWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAA4QAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVTNOR1l0TlRJMk1TMDJOREV3TFdRNE9UVXRPVFF3WldaaVlXUTJNakpqSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/field-goals/2024/post/all/kickingfgmade/desc',
'https://www.nfl.com/stats/player-stats/category/field-goals/2024/REG/all/kickingfgmade/DESC?aftercursor=AAAAGQAAABlANQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNU0lzSWpNeU1EQTBNalUxTFRVME1EWXRNVFkwTWkwek56UTFMVGMxWTJVMVpHVmpNRFF5TWlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/kickoffs/2024/post/all/kickofftotal/desc',
'https://www.nfl.com/stats/player-stats/category/kickoffs/2024/REG/all/kickofftotal/DESC?aftercursor=AAAAGQAAABlATwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJNaUlzSWpNeU1EQTBNalUxTFRVME1EWXRNVFkwTWkwek56UTFMVGMxWTJVMVpHVmpNRFF5TWlJc0lqSXdNalFpWFgwPQ==',
'https://www.nfl.com/stats/player-stats/category/kickoffs/2024/REG/all/kickofftotal/DESC?aftercursor=AAAAMgAAADJAGAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJJaXdpTXpJd01EVXdOREV0TlRRME9TMDRPVEl3TFdNeU1UVXRZemMxTjJabVlqQm1NVGxsSWl3aU1qQXlOQ0pkZlE9PQ==',
'https://www.nfl.com/stats/player-stats/category/kickoff-returns/2024/post/all/kickreturnsaverageyards/desc',
'https://www.nfl.com/stats/player-stats/category/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAAGQAAABlAPzMzMzMzMzFleUp6WldGeVkyaEJablJsY2lJNld5SXpNUzR5SWl3aU16SXdNRFJrTkRrdE5UUXpOeTAyTmpVekxXUm1ObU10Tm1VNVpqVTNNalV6WmpNM0lpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAAMgAAADJAO9R64UeuFDFleUp6WldGeVkyaEJablJsY2lJNld5SXlOeTQ0TXlJc0lqTXlNREEwWWpRMUxUUmxOakV0TkRZeU5TMHhPVFpsTFdZM1pUWTBaRE0wWkRrNE5DSXNJakl3TWpRaVhYMD0=',
'https://www.nfl.com/stats/player-stats/category/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAASwAAAEtAOlR64UeuFDFleUp6WldGeVkyaEJablJsY2lJNld5SXlOaTR6TXlJc0lqTXlNREEwTWpReExUUmlNak10TWpVeU1pMWxZbVUyTFRSaU5qRXpaRE01TkdKbE1DSXNJakl3TWpRaVhYMD0=',
'https://www.nfl.com/stats/player-stats/category/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAAZAAAAGRAOIAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlOQzQxSWl3aU16SXdNRFE0TkRFdE5USXpOaTB6T0RjeExUSm1NakV0TVRKbE5EYzNNVFl4T1RRMElpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAAfQAAAHhANQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNUzR3SWl3aU16SXdNRFUwTkRrdE5UQTJNaTA0TlRVM0xUUTJNbU10TmpSak5ESmpOalUzWTJJeklpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/punts/2024/post/all/puntingaverageyards/desc',
'https://www.nfl.com/stats/player-stats/category/punts/2024/REG/all/puntingaverageyards/DESC?aftercursor=AAAAGQAAABlAR0AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBOaTQxSWl3aU16SXdNRFUzTlRJdE5EazJOQzA1T0RBNUxUZzBabVl0TXprelpERmhOR1EwTkRreElpd2lNakF5TkNKZGZRPT0=',
'https://www.nfl.com/stats/player-stats/category/punt-returns/2024/post/all/puntreturnsaverageyards/desc',
'https://www.nfl.com/stats/player-stats/category/punt-returns/2024/REG/all/puntreturnsaverageyards/DESC?aftercursor=AAAAGQAAABlAJKPXCj1wpDFleUp6WldGeVkyaEJablJsY2lJNld5SXhNQzR6TWlJc0lqTXlNREEwTVRVMUxUVXpOek10T0RBd05DMDFabVl6TFRVeVpqRmpNbUkwWm1Jd1lpSXNJakl3TWpRaVhYMD0=',
'https://www.nfl.com/stats/player-stats/category/punt-returns/2024/REG/all/puntreturnsaverageyards/DESC?aftercursor=AAAAMgAAADJAIAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRMakFpTENJek1qQXdOR0UxTlMwMU16Z3hMVEkzTXpZdE5qazBNQzB5WmpWak1tRXdPREl6WWpjaUxDSXlNREkwSWwxOQ==',
'https://www.nfl.com/stats/player-stats/category/punt-returns/2024/REG/all/puntreturnsaverageyards/DESC?aftercursor=AAAASwAAAEYAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdMakFpTENJek1qQXdOVE0wWXkwME1UVXdMVEF3TnpjdE16RTFaaTAwTjJObVpqSTROemRpTWpNaUxDSXlNREkwSWwxOQ==']

HEADERS = {'Uer-Agent': 'Mozilla/5.0'}

In [29]:
def get_page_content(url, headers):
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup

In [31]:
# Lista para armazenar as URLs dos jogadores e das imagens corrigidas
url_players = []

# Loop sobre todas as URLs principais
for url in URLs:
    # Obter o conteúdo da página para a URL atual
    soup = get_page_content(url, HEADERS)

    # Localizar a tabela de jogadores na página
    table = soup.find('table', {'class': 'd3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable'})

    # Iterar sobre todas as linhas que contêm os links para os jogadores
    for row in table.find_all('a', {'class': 'd3-o-player-fullname nfl-o-cta--link'}):
      # Adicionar o link de cada jogador à lista
      url_players.append(row['href'])

ChunkedEncodingError: ('Connection broken: IncompleteRead(29297 bytes read, 21367 more expected)', IncompleteRead(29297 bytes read, 21367 more expected))

In [22]:
url_players = pd.unique(pd.Series(url_players))

In [26]:
# Configuração das categorias e seus respectivos campos
categories_config = {
    'Passing': [
        'team', 'games', 'attempts', 'completions', 'pass_completion_percentage',
        'yards', 'average', 'long_gain', 'touchdowns', 'interceptions',
        'first_downs', 'first_downs_percentage', 'pass_20_yards_plus',
        'sacks', 'sacks_yards', 'rate'
    ],
    'Rushing': [
        'team', 'games', 'attempts', 'yards', 'average', 'long_gain',
        'touchdowns', 'first_downs', 'first_downs_percentage',
        'rushes_20_yards_plus', 'rushes_40_yards_plus', 'fumbles'
    ],
    'Fumbles': [
        'team', 'games', 'fumbles', 'fumbles_lost', 'forced_fumbles',
        'own_fumbles_recoveries', 'opponent_fumbles_recoveries', 'touchdowns'
    ],
    'Defense': [
        'team', 'games', 'total', 'solo', 'assisted_tackle', 'sacks',
        'safeties', 'pass_defended', 'interceptions', 'touchdowns',
        'yards', 'average', 'long_gain'
    ],
    'Receiving': [
        'team', 'games', 'receptions', 'yards', 'average', 'long_gain',
        'touchdowns', 'first_downs', 'first_downs_percentage',
        'reception_20_yards_plus', 'receptions_40_yards_plus'
    ],
    'Kicking': [
        'team', 'games', 'kick_1_19', 'kick_20_29', 'kick_30_39',
        'kick_40_49', 'kick_50_59', 'kick_60_plus', 'field_goals_made',
        'field_goals_attempts', 'field_goals_percentage'
    ],
    'Punting': [
        'team', 'games', 'punts', 'yards', 'long_gain', 'average',
        'blocked', 'returns', 'return_yards', 'in_20_yards_line',
        'net_punting_average'
    ],
    'Kick Return': [
        'team', 'games', 'returns', 'yards', 'average', 'long_gain',
        'touchdowns', 'returns_20_yards_plus', 'returns_40_yards_plus',
        'fair_catches', 'fumbles'
    ],
    'Punt Return': [
        'team', 'games', 'returns', 'yards', 'average', 'long_gain',
        'touchdowns', 'returns_20_yards_plus', 'returns_40_yards_plus',
        'fair_catches', 'fumbles'
    ]
}

# Diretório para salvar os arquivos
output_dir = "player_stats"
os.makedirs(output_dir, exist_ok=True)

# Função para salvar dados em arquivos CSV
def save_to_csv(category, fields, data):
    file_path = os.path.join(output_dir, f"{category}.csv")
    file_exists = os.path.isfile(file_path)

    fields_with_meta = ['player_id', 'player_name'] + fields

    existing_data = set()
    if file_exists:
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                unique_key = tuple(row.get(field, '') for field in fields_with_meta)
                existing_data.add(unique_key)

    new_data = []
    for row in data:
        unique_key = tuple(row[field] for field in fields_with_meta)
        if unique_key not in existing_data:
            new_data.append(row)

    if new_data:
        with open(file_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=fields_with_meta)
            if not file_exists:
                writer.writeheader()
            writer.writerows(new_data)

def extract_stats(content, category, fields, player_id, player_name):
    stats_list = []

    if category in content.text:
        table = content.find_next('table')
        rows = table.find_all('tr')

        for row in rows:
            if '2024' in row.text:
                infos = row.find_all('td')
                if len(infos) > len(fields):
                    stats = {field: infos[i + 1].text.strip() if i + 1 < len(infos) else '' for i, field in enumerate(fields)}
                    stats['player_id'] = player_id
                    stats['player_name'] = player_name
                    stats_list.append(stats)

    return stats_list

# Loop principal para processar as URLs dos jogadores
counter = 0
for url in url_players:
    player_id = url.split('/players/')[1].strip('/')
    soup = get_page_content(f'https://www.nfl.com{url}stats/career', HEADERS)
    player_name = soup.find('h1', {'class': 'nfl-c-player-header__title'}).text.strip()

    # Extrair a posição do jogador
    position_tag = soup.find('span', {'class': 'nfl-c-player-header__position'})
    player_position = position_tag.text.strip() if position_tag else 'Unknown'

    # Extrair o time do jogador
    team_tag = soup.find_all('td')
    player_team = team_tag[1].text.strip() if team_tag else 'Unknown'

    # Extrair a URL da imagem do jogador
    img_tag = soup.find('img', {'class': 'img-responsive'})
    if img_tag and 'src' in img_tag.attrs:
        player_image_url = img_tag['src'].replace("t_lazy", "t_player_profile_landscape")
    else:
        player_image_url = 'Unknown'

    counter += 1
    print(counter, '-', player_id)

    # Processar estatísticas
    div_table = soup.find_all('div', {'class': 'nfl-o-roster'})

    for content in div_table:
        for category, fields in categories_config.items():
            stats = extract_stats(content, category, fields, player_id, player_name)
            if stats:
                save_to_csv(category, fields, stats)


1 - joe-burrow
2 - jared-goff
3 - baker-mayfield
4 - geno-smith
5 - sam-darnold
6 - lamar-jackson
7 - patrick-mahomes
8 - aaron-rodgers
9 - justin-herbert
10 - brock-purdy
11 - kyler-murray
12 - bo-nix
13 - matthew-stafford
14 - josh-allen
15 - c-j-stroud
16 - jayden-daniels
17 - caleb-williams
18 - kirk-cousins
19 - jordan-love
20 - jalen-hurts
21 - tua-tagovailoa
22 - russell-wilson
23 - bryce-young
24 - drake-maye
25 - derek-carr
26 - jameis-winston
27 - will-levis
28 - daniel-jones-x5302
29 - trevor-lawrence
30 - gardner-minshew
31 - dak-prescott
32 - cooper-rush
33 - anthony-richardson
34 - joe-flacco
35 - mac-jones
36 - aidan-o-connell
37 - mason-rudolph
38 - spencer-rattler
39 - deshaun-watson
40 - justin-fields
41 - drew-lock
42 - andy-dalton
43 - tyler-huntley
44 - jacoby-brissett
45 - michael-penix-jr
46 - malik-willis
47 - desmond-ridder
48 - dorian-thompson-robinson
49 - marcus-mariota
50 - joshua-dobbs
51 - jimmy-garoppolo
52 - tanner-mckee
53 - kenny-pickett
54 - tim-boyl